# Constrained Double Pendulum Simulation
This notebook simulates a double pendulum using `DistanceConstraint` to model rigid rods.

In [1]:
import sys
import os
import math
import time
from pythreejs import *
import numpy as np

# Adjust path to find the module
build_dir = "/Users/jimmydanielvacareanu/Desktop/NSSC1/nssc1-team07/build/mechsystem"
sys.path.insert(0, build_dir)

try:
    from mass_spring import *
except ImportError:
    print("Error: Could not import mass_spring module. Make sure you have built the project.")


## System Setup
We set up two masses connected by rigid constraints.

In [2]:
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

# Create masses and fix
# Start horizontally to avoid singularities
# Mass 1 at (1, 0, 0)
m1 = mss.add(Mass(1.0, [1.0, 0.0, 0.0]))

# Mass 2 at (2, 0, 0)
m2 = mss.add(Mass(1.0, [2.0, 0.0, 0.0]))

# Fix at origin
f1 = mss.add(Fix((0, 0, 0)))

# Setup constraints
# Constraint 1: Fix -> m1 (Length 1)
mss.add(DistanceConstraint(1, (f1, m1)))
# Constraint 2: m1 -> m2 (Length 1)
mss.add(DistanceConstraint(1, (m1, m2)))

print(f"Number of masses: {len(mss.masses)}")
print(f"Number of constraints: {len(mss.constraints)}")

Number of masses: 2
Number of constraints: 2


## Floating Mass Check
Verify that all masses are connected.

In [3]:
connected_masses = set()
for i, c in enumerate(mss.constraints):
    for connector in c.connectors:
        if connector.type == Connector.MASS:
            connected_masses.add(connector.nr)

all_mass_indices = set(range(len(mss.masses)))
unconnected_masses = all_mass_indices - connected_masses

if unconnected_masses:
    print(f"WARNING: Floating masses detected: {unconnected_masses}")
else:
    print("OK: All masses are connected.")

OK: All masses are connected.


## Visualization Setup
Using `pythreejs` to render the scene.

In [4]:
mass_meshes = []
for m in mss.masses:
    mass_meshes.append(
        Mesh(
            SphereBufferGeometry(0.1, 16, 16),
            MeshStandardMaterial(color="red"),
            position=m.pos,
        )
    )

fix_meshes = []
for f in mss.fixes:
    fix_meshes.append(
        Mesh(
            BoxBufferGeometry(0.2, 0.2, 0.2),
            MeshStandardMaterial(color="blue"),
            position=f.pos,
        )
    )

constraint_pos = []
for c in mss.constraints:
    pA = mss[c.connectors[0]].pos
    pB = mss[c.connectors[1]].pos
    constraint_pos.append([pA, pB])

constraint_geo = LineSegmentsGeometry(positions=constraint_pos)
constraint_mat = LineMaterial(linewidth=2, color="cyan")
constraints_line = LineSegments2(constraint_geo, constraint_mat)

view_width = 600
view_height = 400
camera = PerspectiveCamera(position=[0, 2, 5], aspect=view_width/view_height, look_at=[0, -1, 0])
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight(intensity=0.5)

scene = Scene(children=[*mass_meshes, *fix_meshes, constraints_line, camera, key_light, ambient_light, AxesHelper(1)])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=view_width, height=view_height)
renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(0.0, 2.0, 5.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, …

## Simulation Loop
Running the simulation using `mss.simulate`.

In [ ]:
# Simulation parameters
dt = 0.00001 # Small time step for stability
steps_per_frame = 1000 # 0.01s per frame

print("Starting simulation...")
for i in range(10000):
    mss.simulate(0.01, steps_per_frame)
    
    # Update visualization
    for m, mesh in zip(mss.masses, mass_meshes):
        mesh.position = tuple(m.pos)
        
    new_constraint_pos = []
    for c in mss.constraints:
        pA = mss[c.connectors[0]].pos
        pB = mss[c.connectors[1]].pos
        new_constraint_pos.append([pA, pB])
    
    constraints_line.geometry = LineSegmentsGeometry(positions=new_constraint_pos)
    
    time.sleep(0.01)

Starting simulation...
